In [21]:
!pip install tabulate numpy


[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
#sme predefinitions to make takings actions easier
L=0
R=1
U=2
D=3
S=4

In [23]:
###_MAKING GRID

In [24]:
from __future__ import annotations
import numpy as np
from tabulate import tabulate
from typing import List, Tuple, Iterable, Set

Action = int  # alias for readability (0‑4)
State  = int  # flattened index

# Displacement vectors for actions: L, R, U, D, Stay
_DRDC = [ (0, -1), (0, 1), (-1, 0), (1, 0), (0, 0) ]

HOLE_REWARD  = -1.0
GOAL_REWARD  = +1.0
STEP_REWARD  =  0.0

class GridWorld:
    """Deterministic tabular environment suitable for DP algorithms."""
    def __init__(self,
                 rows: int,
                 cols: int,
                 start: Tuple[int, int],
                 goal: Tuple[int, int],
                 holes: Iterable[Tuple[int, int]] = ()):        
        self.rows, self.cols = rows, cols
        self.nS, self.nA = rows * cols, 5

        # Helper lambdas ------------------------------------------------------
        self._to_id   = lambda rc: rc[0] * self.cols + rc[1]
        self._to_rc   = lambda s: divmod(s, self.cols)

        # Convert & store special cells --------------------------------------
        self.start: State = self._to_id(start)
        self.goal:  State = self._to_id(goal)
        self.holes: Set[State] = {self._to_id(h) for h in holes if h != goal and h != start}

        # Tables --------------------------------------------------------------
        self.next_state = np.zeros((self.nS, self.nA), dtype=np.int32)
        self.reward     = np.zeros((self.nS, self.nA), dtype=np.float32)
        self._build_transition_tables()

    # ---------------------------------------------------------------------
    def _build_transition_tables(self):
        for s in range(self.nS):
            r, c = self._to_rc(s)
            for a, (dr, dc) in enumerate(_DRDC):
                nr, nc = r + dr, c + dc
                # Off‑grid? => stay, reward -1
                if nr < 0 or nr >= self.rows or nc < 0 or nc >= self.cols:
                    self.next_state[s, a] = s
                    self.reward[s, a]     = HOLE_REWARD  # same as off‑grid penalty
                    continue

                s_next = self._to_id((nr, nc))
                self.next_state[s, a] = s_next

                if s_next == self.goal:
                    self.reward[s, a] = GOAL_REWARD
                elif s_next in self.holes:
                    self.reward[s, a] = HOLE_REWARD
                else:
                    self.reward[s, a] = STEP_REWARD

    # ------------------------------------------------------------------ env API
    def reset(self) -> State:
        return self.start

    def step(self, s: State, a: Action) -> Tuple[State, float]:
        """Return (next_state, reward).  No done flag because episodes never end."""
        return int(self.next_state[s, a]), float(self.reward[s, a])

    # -------------------------------------------------------------- visualisation
    def render(self, agent_state: State | None = None):
        symbols = []
        for s in range(self.nS):
            if s == agent_state:
                symbols.append("A")
            elif s == self.start:
                symbols.append("S")
            elif s == self.goal:
                symbols.append("G")
            elif s in self.holes:
                symbols.append("H")
            else:
                symbols.append("·")
        # reshape & draw ------------------------------------------------------
        rows = [ symbols[i:i+self.cols] for i in range(0, self.nS, self.cols) ]
        print(tabulate(rows, tablefmt="grid"))

# ---------------------------------------------------------------------------
# Utility to build a random instance ----------------------------------------
# ---------------------------------------------------------------------------

def make_random_grid(rows: int,
                     cols: int,
                     n_holes: int = 0,
                     seed: int | None = None) -> GridWorld:
    rng = np.random.default_rng(seed)
    all_cells = [(r, c) for r in range(rows) for c in range(cols)]
    start = tuple(map(int, rng.choice(all_cells)))
    remaining = [cell for cell in all_cells if cell != start]
    goal  = tuple(map(int, rng.choice(remaining)))
    remaining.remove(goal)
    holes = [tuple(map(int, h)) for h in rng.choice(remaining, size=min(n_holes, len(remaining)), replace=False)]
    return GridWorld(rows, cols, start, goal, holes)

def show_table(mat, headers, title=""):
    rows = [[s, *mat[s]] for s in range(mat.shape[0])]
    print(f"\n{title}")
    print(tabulate(rows,
                   headers=["state", *headers],
                   floatfmt=".1f",
                   tablefmt="grid"))

    


In [113]:
env = make_random_grid(7, 7, n_holes=8, seed=40)
env.render()
    


+---+---+---+---+---+---+---+
| H | H | H | · | · | · | · |
+---+---+---+---+---+---+---+
| · | · | · | · | · | · | · |
+---+---+---+---+---+---+---+
| · | · | · | · | H | · | · |
+---+---+---+---+---+---+---+
| H | · | · | · | · | · | S |
+---+---+---+---+---+---+---+
| · | H | · | · | · | H | · |
+---+---+---+---+---+---+---+
| · | G | · | · | · | · | · |
+---+---+---+---+---+---+---+
| H | · | · | · | · | · | · |
+---+---+---+---+---+---+---+


In [114]:
env.reward[0][1]

-1.0

In [115]:
acts = ["L", "R", "U", "D", "S"]
show_table(env.reward,      acts, "Reward table")
show_table(env.next_state,  acts, "Next-state table")


Reward table
+---------+------+------+------+------+------+
|   state |    L |    R |    U |    D |    S |
+=========+======+======+======+======+======+
|       0 | -1.0 | -1.0 | -1.0 |  0.0 | -1.0 |
+---------+------+------+------+------+------+
|       1 | -1.0 | -1.0 | -1.0 |  0.0 | -1.0 |
+---------+------+------+------+------+------+
|       2 | -1.0 |  0.0 | -1.0 |  0.0 | -1.0 |
+---------+------+------+------+------+------+
|       3 | -1.0 |  0.0 | -1.0 |  0.0 |  0.0 |
+---------+------+------+------+------+------+
|       4 |  0.0 |  0.0 | -1.0 |  0.0 |  0.0 |
+---------+------+------+------+------+------+
|       5 |  0.0 |  0.0 | -1.0 |  0.0 |  0.0 |
+---------+------+------+------+------+------+
|       6 |  0.0 | -1.0 | -1.0 |  0.0 |  0.0 |
+---------+------+------+------+------+------+
|       7 | -1.0 |  0.0 | -1.0 |  0.0 |  0.0 |
+---------+------+------+------+------+------+
|       8 |  0.0 |  0.0 | -1.0 |  0.0 |  0.0 |
+---------+------+------+------+------+------+

In [116]:
discount = 0.9
def init_v(env: GridWorld):
    return np.zeros(env.nS)

def init_pi(env: GridWorld):
    pi = np.empty(env.nS)
    for state in range(env.nS):
        pi[state] = S
    return pi
        
        
def value_iteration(env: GridWorld):
    #initialize current policy: Stay everywhere    
    # pi is a nSx2 grid: for each state, what action we take. In this case, (0,0) for all
    #initialize v = 0 for all S
    # now make a q table with nSxnA
    #loop over s
        #for every a:
            #q[s,a] = immediate reward for the action from reward table + discount rate x v(next state for that action)
        #take maximum a and then set pi[s] = a
        #then set v[s] to the return of chosing this max a

    pi = init_pi(env)
    v_table = init_v(env)
    q_table = np.full((env.nS, env.nA), 0, dtype=float)
    diff = float('inf')
    while diff>0.01:
        v_table_new = np.copy(v_table)
        for state in range (env.nS):
            max_reward = float('-inf')
            for action in range (env.nA):
                qvalue = env.reward[state][action] + discount*v_table[env.next_state[state][action]]
                q_table[state][action] = qvalue
                if qvalue>=max_reward:
                    max_reward = qvalue
                    pi[state] = action
            v_table_new[state] = max_reward
        diff = np.sum(np.absolute(v_table_new-v_table))
        v_table = np.copy(v_table_new)

    ##Recreate path
    # path = recreate_path(pi, env)
    
    
    
    return pi, v_table, q_table

def recreate_path(pi: np.array, env: GridWorld):
    path = np.zeros((env.rows,env.cols))
    ctr = 0
    for i in range(env.rows):
        for j in range (env.cols):
            path[i][j] = pi[ctr]
            ctr+=1
        
    return path
        

In [117]:
###Policy iteration

In [118]:
env.render()

+---+---+---+---+---+---+---+
| H | H | H | · | · | · | · |
+---+---+---+---+---+---+---+
| · | · | · | · | · | · | · |
+---+---+---+---+---+---+---+
| · | · | · | · | H | · | · |
+---+---+---+---+---+---+---+
| H | · | · | · | · | · | S |
+---+---+---+---+---+---+---+
| · | H | · | · | · | H | · |
+---+---+---+---+---+---+---+
| · | G | · | · | · | · | · |
+---+---+---+---+---+---+---+
| H | · | · | · | · | · | · |
+---+---+---+---+---+---+---+


In [123]:
def evaluate_policy(pi: np.ndarray, env: GridWorld, 
                    error: float = 1e-10) -> np.ndarray:
    V = np.zeros(env.nS)
    while True:
        delta = 0.0
        for s in range(env.nS):
            a      = int(pi[s])
            v_new  = env.reward[s, a] + discount * V[env.next_state[s, a]]
            delta  = max(delta, abs(v_new - V[s]))
            V[s]   = v_new
        if delta < error:
            break
    return V


def policy_iteration(env: GridWorld):
    pi = np.zeros(env.nS, dtype=int)      # arbitrary start, say all 0 = Left
    while True:
        V = evaluate_policy(pi, env)
        policy_stable = True
        for s in range(env.nS):
            # one-step look-ahead for all actions
            q_sa = env.reward[s] + discount * V[env.next_state[s]]
            best_a = int(np.argmax(q_sa))
            if best_a != pi[s]:
                policy_stable = False
                pi[s] = best_a
        if policy_stable:
            break
    return pi, V


In [124]:
p, v = policy_iteration(env)

In [125]:
recreate_path(p, env)

array([[3., 3., 3., 3., 3., 3., 3.],
       [1., 3., 3., 3., 0., 3., 3.],
       [1., 3., 3., 3., 3., 3., 3.],
       [3., 3., 3., 3., 3., 0., 3.],
       [3., 3., 3., 3., 3., 3., 3.],
       [1., 4., 0., 0., 0., 0., 0.],
       [1., 2., 0., 0., 0., 0., 0.]])

In [126]:
env.render()

+---+---+---+---+---+---+---+
| H | H | H | · | · | · | · |
+---+---+---+---+---+---+---+
| · | · | · | · | · | · | · |
+---+---+---+---+---+---+---+
| · | · | · | · | H | · | · |
+---+---+---+---+---+---+---+
| H | · | · | · | · | · | S |
+---+---+---+---+---+---+---+
| · | H | · | · | · | H | · |
+---+---+---+---+---+---+---+
| · | G | · | · | · | · | · |
+---+---+---+---+---+---+---+
| H | · | · | · | · | · | · |
+---+---+---+---+---+---+---+
